code-3.py

In [6]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

# atur parameter
theta = np.pi/4
r = 0.95
err = 10^(-5)

npoint = 20
dim = 3
iter = 0
iter_points = {}

iter_max = 100


def generate_points(low,high,dim,npoint):
    tupple = {}
    for i in range (dim):
        tupple[f'basis {i+1}'] = np.random.uniform(low,high, npoint)
    print(tupple)
    points = np.column_stack((tuple(tupple[f'basis {i+1}'] for i in range(dim))))
    return points

def objective_function(x): # x:tuple n-dimension
    f = 0
    """Schwefel"""
    for i in range (len(x)):
        sum_sq = 0
        for j in range (i+1):
            sum_sq += x[j]
        f += sum_sq**2 
    """2^n Minima"""
    # for i in range (len(x)):
    #     f += (x[i]**4-16*x[i]**2+5*x[i])
    """Rastrigin"""
    # for i in range (len(x)):
    #     f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)

    return f

def generate_Rij(i,j,dim,theta):
    Rn_ij= np.eye(dim)
    Rn_ij[i-1,i-1] = np.cos(theta)
    Rn_ij[i-1,j-1] = -np.sin(theta)
    Rn_ij[j-1,i-1] = np.sin(theta)
    Rn_ij[j-1,j-1] = np.cos(theta)
    return Rn_ij

def generate_Rn(dim,theta):
    Rn = np.eye(dim)
    for i in range(0,dim):
        for j in range (0,i+1):
            product = np.eye(dim)
            product *= generate_Rij(dim-i-1,dim+1-j-1,dim,theta)
        Rn *= product
    return Rn

iter_points[iter] = generate_points(-5,5,dim,npoint)
iter_points[iter]


{'basis 1': array([ 0.48813504,  2.15189366,  1.02763376,  0.44883183, -0.76345201,
        1.45894113, -0.62412789,  3.91773001,  4.63662761, -1.16558481,
        2.91725038,  0.2889492 ,  0.68044561,  4.25596638, -4.28963942,
       -4.128707  , -4.79781603,  3.32619846,  2.78156751,  3.70012148]), 'basis 2': array([ 4.78618342,  2.99158564, -0.38520638,  2.80529176, -3.81725574,
        1.39921021, -3.56646713,  4.44668917,  0.21848322, -0.8533806 ,
       -2.35444388,  2.74233689, -0.43849668,  0.68433949, -4.812102  ,
        1.17635497,  1.12095723,  1.16933997,  4.43748079,  1.81820299]), 'basis 3': array([-1.40492099, -0.62968046,  1.97631196, -4.39774528,  1.66766715,
        1.7063787 , -2.89617439, -3.71073702, -1.84571649, -1.36289229,
        0.7019677 , -0.61398487,  4.88373838, -3.97955189, -2.91123244,
       -3.38690482,  1.53108325, -2.46708397, -0.33689227, -2.55574408])}


array([[ 0.48813504,  4.78618342, -1.40492099],
       [ 2.15189366,  2.99158564, -0.62968046],
       [ 1.02763376, -0.38520638,  1.97631196],
       [ 0.44883183,  2.80529176, -4.39774528],
       [-0.76345201, -3.81725574,  1.66766715],
       [ 1.45894113,  1.39921021,  1.7063787 ],
       [-0.62412789, -3.56646713, -2.89617439],
       [ 3.91773001,  4.44668917, -3.71073702],
       [ 4.63662761,  0.21848322, -1.84571649],
       [-1.16558481, -0.8533806 , -1.36289229],
       [ 2.91725038, -2.35444388,  0.7019677 ],
       [ 0.2889492 ,  2.74233689, -0.61398487],
       [ 0.68044561, -0.43849668,  4.88373838],
       [ 4.25596638,  0.68433949, -3.97955189],
       [-4.28963942, -4.812102  , -2.91123244],
       [-4.128707  ,  1.17635497, -3.38690482],
       [-4.79781603,  1.12095723,  1.53108325],
       [ 3.32619846,  1.16933997, -2.46708397],
       [ 2.78156751,  4.43748079, -0.33689227],
       [ 3.70012148,  1.81820299, -2.55574408]])

In [2]:
Rn = generate_Rn(dim,theta)
Sn = r*Rn
error_max = 10**(-5)

def minimize(set_of_points):
    z = []
    z_min = 10**100
    for i in range (len(set_of_points)):
        z.append(objective_function(set_of_points.T)[i])
        if z[i]<z_min:
            z_min = z[i]
            idx_min = i
    x_min = set_of_points[idx_min]
    return z_min,idx_min,x_min

def update_point(set_of_points):
    (z_star,idx_star,x_star) = minimize(set_of_points)
    new_set_of_points = np.copy(set_of_points)
    for i in range (len(new_set_of_points)):
        # perkalian matriks
        poin = np.dot(Sn,set_of_points[i].reshape(-1,1)) - np.dot((Sn-np.identity(dim)),x_star.reshape(-1,1))
        new_set_of_points[i] = poin.T
    return new_set_of_points

def iter_error(set_of_points,iter):
    err = 0
    for i in range (npoint):
        diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))
        if diff>err:
            err = diff
    return err

while iter <= iter_max :
    iter_points[iter+1] = update_point(iter_points[iter])
    if iter_error(iter_points,iter+1) < error_max:
        break
    iter += 1

C:\Users\ACER\AppData\Local\Temp\ipykernel_16452\1594175278.py:28: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))


In [5]:
import numpy as np
import sobol_seq

# atur parameter
theta = np.pi/4
r = 0.95
err = 10^(-5)

npoint = 20
dim = 3

iter_max = 100
def spiralopt(low_point,high_point, dim, npoint,r = 0.95,theta=np.pi/4, iter_max=100, error_max = 10**(-5),random=0, show_err=False, show_objective_function=False):
    np.random.seed(random)
    iter_points = {}
    iter = 0
    def generate_points(dim,npoint,low=-10,high=10):
        if type(low) != type(high):
            raise TypeError('The type of "low" and "high" should be the same.')
        if type(low) == int:
            boundaries = [(low,high) for _ in range (dim)]
        elif type(low) == list or type(low) == np.ndarray:
            if len(low) != len(high):
                raise TypeError('The length of "low" and "high" should be the same.')
            else:
                boundaries = [(low[i],high[i]) for i in range (len(low))]

        # Generate Sobol sequence points
        sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

        # Scale the Sobol points to fit within the specified boundaries
        scaled_points = []
        for i in range(dim):
            a, b = boundaries[i]
            scaled_dim = a + sobol_points[:, i] * (b - a)
            scaled_points.append(scaled_dim)

        # Transpose the scaled points to get points per dimension
        scaled_points = np.array(list(map(list, zip(*scaled_points))))
        return scaled_points

    def objective_function(x): # x:tuple n-dimension
        f = 0
        """Schwefel"""
        for i in range (len(x)):
            sum_sq = 0
            for j in range (i+1):
                sum_sq += x[j]
            f += sum_sq**2 
        """2^n Minima"""
        # for i in range (len(x)):
        #     f += (x[i]**4-16*x[i]**2+5*x[i])
        """Rastrigin"""
        # for i in range (len(x)):
        #     f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)

        return f

    def generate_Rij(i,j,dim,theta):
        Rn_ij= np.eye(dim)
        Rn_ij[i-1,i-1] = np.cos(theta)
        Rn_ij[i-1,j-1] = -np.sin(theta)
        Rn_ij[j-1,i-1] = np.sin(theta)
        Rn_ij[j-1,j-1] = np.cos(theta)
        return Rn_ij

    def generate_Rn(dim,theta):
        Rn = np.eye(dim)
        for i in range(0,dim):
            for j in range (0,i+1):
                product = np.eye(dim)
                product *= generate_Rij(dim-i-1,dim+1-j-1,dim,theta)
            Rn *= product
        return Rn

    iter_points[iter] = generate_points(dim,npoint,low_point,high_point)

    Rn = generate_Rn(dim,theta)
    Sn = r*Rn

    def minimize(set_of_points):
        z = []
        z_min = 10**100
        for i in range (len(set_of_points)):
            z.append(objective_function(set_of_points.T)[i])
            if z[i]<z_min:
                z_min = z[i]
                idx_min = i
        x_min = set_of_points[idx_min]
        return z_min,idx_min,x_min

    def update_point(set_of_points):
        (z_star,idx_star,x_star) = minimize(set_of_points)
        new_set_of_points = np.copy(set_of_points)
        for i in range (len(new_set_of_points)):
            # perkalian matriks
            poin = np.dot(Sn,set_of_points[i].reshape(-1,1)) - np.dot((Sn-np.identity(dim)),x_star.reshape(-1,1))
            new_set_of_points[i] = poin.T
        return new_set_of_points

    def iter_error(set_of_points,iter):
        err = 0
        for i in range (npoint):
            diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))
            if diff>err:
                err = diff
        return err

    while iter <= iter_max :
        iter_points[iter+1] = update_point(iter_points[iter])
        error = iter_error(iter_points,iter+1)
        if error < error_max:
            break
        iter += 1

    return_points = [iter_points[iter][:,i].mean() for i in range(dim)]
    if show_err == True:
        print(error)
    obj_fun = objective_function(iter_points[iter].T).mean()
    if show_objective_function ==True:
        print(obj_fun)
    return return_points, obj_fun


opt_point, opt_val = [],[]
for i,rand in enumerate(np.random.randint(100,size=30)):
    set_of_points,optimized_value = spiralopt(low_point=-5,high_point=5, dim=dim, npoint=npoint, iter_max=iter_max, random=rand, show_err=False, show_objective_function=False)
    opt_point.append(set_of_points)
    opt_val.append(optimized_value)
opt_point = np.array(opt_point)
opt_val = np.array(opt_val)
print(opt_point)
print(opt_val)

C:\Users\ACER\AppData\Local\Temp\ipykernel_1640\2794777362.py:104: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))


[[-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-08 -1.18379092e-07]
 [-1.18379092e-07 -2.36758184e-

In [2]:
print(opt_val.mean())
print(opt_val.std())

7.459724537290451e-11
3.877409121342317e-26


In [9]:
spiralopt(low_point=-5,high_point=5, dim=3, npoint=20, iter_max=100, random=0, show_err=False, show_objective_function=False)

C:\Users\ACER\AppData\Local\Temp\ipykernel_4800\308920542.py:85: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))


([-2.3251056077834362, -2.664763913455232, -3.0219448696312146],
 -223.24081192944226)

In [5]:
import numpy as np
import sobol_seq

def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points

def objective_function(x): # x:tuple n-dimension
    f = 0
    """Schwefel"""
    # for i in range (len(x)):
    #     sum_sq = 0
    #     for j in range (i+1):
    #         sum_sq += x[j]
    #     f += sum_sq**2 
    """2^n Minima"""
    for i in range (len(x)):
        f += (x[i]**4-16*x[i]**2+5*x[i])
    """Rastrigin"""
    # for i in range (len(x)):
    #     f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)

    return f

def generate_Rij(i,j,dim,theta):
    Rn_ij= np.eye(dim)
    Rn_ij[i-1,i-1] = np.cos(theta)
    Rn_ij[i-1,j-1] = -np.sin(theta)
    Rn_ij[j-1,i-1] = np.sin(theta)
    Rn_ij[j-1,j-1] = np.cos(theta)
    return Rn_ij

def generate_Rn(dim,theta):
    Rn = np.eye(dim)
    for i in range(0,dim):
        for j in range (0,i+1):
            product = np.eye(dim)
            product *= generate_Rij(dim-i-1,dim+1-j-1,dim,theta)
        Rn *= product
    return Rn

def minimize(set_of_points):
    z = []
    z_min = 10**100
    for i in range (len(set_of_points)):
        z.append(objective_function(set_of_points.T)[i])
        if z[i]<z_min:
            z_min = z[i]
            idx_min = i
    x_min = set_of_points[idx_min]
    return z_min,idx_min,x_min

def update_point(set_of_points,Sn,dim):
    (z_star,idx_star,x_star) = minimize(set_of_points)
    new_set_of_points = np.copy(set_of_points)
    for i in range (len(new_set_of_points)):
        # perkalian matriks
        poin = np.dot(Sn,set_of_points[i].reshape(-1,1)) - np.dot((Sn-np.identity(dim)),x_star.reshape(-1,1))
        new_set_of_points[i] = poin.T
    return new_set_of_points

def iter_error(set_of_points,iter,npoint):
    err = 0
    for i in range (npoint):
        diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))
        if diff>err:
            err = diff
    return err
    
def SpiralOpt(low_point,high_point, dim, npoint,r = 0.95,theta=np.pi/4, iter_max=100, error_max = 10**(-5),random=0, show_err=False, show_objective_function=False):
    np.random.seed(random)
    iter_points = {}
    iter = 0
    iter_points[iter] = generate_points(dim,npoint,low_point,high_point)
    Rn = generate_Rn(dim,theta)
    Sn = r*Rn
    while iter <= iter_max :
        iter_points[iter+1] = update_point(iter_points[iter],Sn,dim)
        error = iter_error(iter_points,iter+1,npoint)
        if error < error_max:
            break
        iter += 1

    return_points = [iter_points[iter][:,i].mean() for i in range(dim)]
    if show_err == True:
        print(error)
    obj_fun = objective_function(iter_points[iter].T).mean()
    if show_objective_function ==True:
        print(obj_fun)
    return return_points, obj_fun

SpiralOpt(low_point=-5,high_point=5, dim=4, npoint=20, iter_max=100, random=100, show_err=False, show_objective_function=False)

# opt_point, opt_val = [],[]
# for i,rand in enumerate(np.random.randint(100,size=30)):
#     set_of_points,optimized_value = SpiralOpt(low_point=-5,high_point=5, dim=4, npoint=20, iter_max=100, random=100, show_err=False, show_objective_function=False)
#     opt_point.append(set_of_points)
#     opt_val.append(optimized_value)
# opt_point = np.array(opt_point)
# opt_val = np.array(opt_val)
# print(opt_point)
# print(opt_val)

C:\Users\ACER\AppData\Local\Temp\ipykernel_13716\2418681515.py:86: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))


([2.4999960934899716,
  -2.499996235544882,
  2.499996093489972,
  -2.499996282896519],
 -243.7497339994706)